In [ ]:
# Import libraries
import glob
import os
import pandas as pd
from shutil import copyfile
import shutil
import glob
import numpy as np
import math
# This is for seeing how long this script takes to be run and can be removed before batch processing
from datetime import datetime

In [ ]:
# Set file path for base directory and BIDS directory
study_dir = '/PATH_TO_BASE_STUDY_DIRECTORY'
base_dir = str(study_dir + '/')
BIDS = str(base_dir + 'derivatives/abcd-hcp-pipeline/')

# Specify which task you are running the pipeline on and for which session
task_name = 'nback'
ses = 'ses-baselineYear1Arm1'

# Create paths for various variables that are called throughout the pipeline
templates = str(base_dir + 'templates/')
MNI_mask = str (templates + 'MNI152_T1_2mm_brain_mask.nii.gz')

# Create variables to decrease lengths of various variables throughout the pipeline
session = str('_' + ses + '_')
fmri_task = str('task-' + task_name)
fmri_task_run = str('task-' + task_name + '_run-')

############
startTime = datetime.now()

In [ ]:
startTime_QC = datetime.now()

for subject in os.listdir(BIDS): 
    sub_run_task = str(subject + session + fmri_task)
    sub_run_basename = str(subject + session + fmri_task_run)
    sub_func_path = str(BIDS + subject + '/' + ses + '/func/')
    volume_run_1 = str(sub_func_path + sub_run_basename + '1_space-MNI_bold.nii.gz')
    volume_run_2 = str(sub_func_path + sub_run_basename + '2_space-MNI_bold.nii.gz')
    
    if os.path.exists(volume_run_1):

        bold_vol = str(sub_func_path + sub_run_basename + '1_space-MNI_bold.nii.gz')
        prefiltered_func = str(sub_func_path + sub_run_basename + '1_space-MNI_bold_prefiltered_func.nii.gz')
        mean_func = str(sub_func_path + sub_run_basename + '1_space-MNI_bold_mean_func.nii.gz')    
        mean_func_mask = str(sub_func_path + sub_run_basename + '1_mean_func_mask.nii.gz')
        mean_func_mask_masked = str(sub_func_path + sub_run_basename + '1_mean_func_mask_masked.nii.gz')

        os.system('fslmaths %s %s -odt float'%(bold_vol, prefiltered_func))
        os.system('fslmaths %s -Tmean %s'%(prefiltered_func, mean_func))
        os.system('fslmaths %s -bin %s'%(mean_func, mean_func_mask))
        os.system('fslmaths %s -mas %s %s'%(mean_func_mask, MNI_mask, mean_func_mask_masked))
        num_vox = os.popen('fslstats %s -V | awk {\'print $1\'}'%(mean_func_mask_masked)).read().rstrip()

        num_vox_val = float(num_vox)
        num_vox_perc = (num_vox_val/228483)*100
        num_vox_perc_round = round(num_vox_perc,3)
        sub_run = str(sub_run_basename + '1')
        d = {'sub_run':sub_run, 'num_vox_perc_round':num_vox_perc_round}
        Index = [0]
        df = pd.DataFrame(data = d, index = Index)
        df.to_csv(sub_func_path + sub_run_basename + '1_brain_coverage.tsv', sep='\t', encoding='utf-8', header=None, index=False)
        
    else:
        print('No run-1 found for:', sub_run_task)

    if os.path.exists(volume_run_2):
        bold_vol = str(sub_func_path + sub_run_basename + '2_space-MNI_bold.nii.gz')
        prefiltered_func = str(sub_func_path + sub_run_basename + '2_space-MNI_bold_prefiltered_func.nii.gz')
        mean_func = str(sub_func_path + sub_run_basename + '2_space-MNI_bold_mean_func.nii.gz')    
        mean_func_mask = str(sub_func_path + sub_run_basename + '2_mean_func_mask.nii.gz')
        mean_func_mask_masked = str(sub_func_path + sub_run_basename + '2_mean_func_mask_masked.nii.gz')

        os.system('fslmaths %s %s -odt float'%(bold_vol, prefiltered_func))
        os.system('fslmaths %s -Tmean %s'%(prefiltered_func, mean_func))
        os.system('fslmaths %s -bin %s'%(mean_func, mean_func_mask))
        os.system('fslmaths %s -mas %s %s'%(mean_func_mask, MNI_mask, mean_func_mask_masked))
        num_vox = os.popen('fslstats %s -V | awk {\'print $1\'}'%(mean_func_mask_masked)).read().rstrip()

        num_vox_val = float(num_vox)
        num_vox_perc = (num_vox_val/228483)*100
        num_vox_perc_round = round(num_vox_perc,3)
        sub_run = str(sub_run_basename + '2')
        d = {'sub_run':sub_run, 'num_vox_perc_round':num_vox_perc_round}
        Index = [0]
        df = pd.DataFrame(data = d, index = Index)
        df.to_csv(sub_func_path + sub_run_basename + '2_brain_coverage.tsv', sep='\t', encoding='utf-8', header=None, index=False)

    else:
        print('No run-2 found for:', sub_run_task)

In [ ]:
for subject in os.listdir(BIDS):
    sub_run_basename = str(subject + session + fmri_task_run)
    sub_func_path = str(BIDS + subject + '/' + ses + '/func/')
    Run1 = str(sub_func_path + sub_run_basename + '1_brain_coverage.tsv')
    Run2 = str(sub_func_path + sub_run_basename + '2_brain_coverage.tsv')
    
    if os.path.exists(Run1):
        print('Brain coverage file for run-1: FOUND')

    else:
        sub_run = str(sub_run_basename + '1')
        d = {'sub_run':sub_run, 'num_vox_perc_round':'n/a'}
        Index = [0]
        df = pd.DataFrame(data = d, index = Index)
        df.to_csv(sub_func_path + sub_run + '_brain_coverage.tsv', sep='\t', encoding='utf-8', header=None, index=False)

    if os.path.exists(Run2):
        print('Brain coverage file for run-2: FOUND')

    else:
        sub_run = str(sub_run_basename + '2')
        d = {'sub_run':sub_run, 'num_vox_perc_round':'n/a'}
        Index = [0]
        df = pd.DataFrame(data = d, index = Index)
        df.to_csv(sub_func_path + sub_run + '_brain_coverage.tsv', sep='\t', encoding='utf-8', header=None, index=False)

for subject in os.listdir(BIDS):
    sub_run_basename = str(subject + session + fmri_task_run)
    sub_func_path = str(BIDS + subject + '/' + ses + '/func/')
    Run1 = str(sub_func_path + sub_run_basename + '1_brain_coverage.tsv')
    Run2 = str(sub_func_path + sub_run_basename + '2_brain_coverage.tsv')

    df1 = pd.read_csv(Run1, sep='\t')
    df2 = pd.read_csv(Run2, sep='\t')
    result = pd.concat([df1, df2], axis=1)
    outfile = str(base_dir + 'QC_files/brain_coverage_info' + session + fmri_task + '.tsv')
    with open(outfile, 'a') as outfile:
        result.to_csv(outfile, sep='\t', header=True, index=False)
        
print('The pipeline took this long to run: ', datetime.now() - startTime)